# 2.1. 用llamaindex开发新人答疑机器人

## 🚅 前言 

LlamaIndex是一个易于使用的工具，可以帮助你快速构建RAG（Retrieval-Augmented Generation）应用。它只需简单的几步，就能让你高效查询和管理数据。通过本教程的学习，你可以轻松加载数据并构建索引，快速得到所需信息。此外，LlamaIndex兼容DashScope API，能够生成高质量的自然语言响应，非常适合初学者和开发者。

## 🍁 课程目标

学完本课程后，你将能够：<br>
- 学会通过LlamaIndex框架调用通义千问模型的方法
- 掌握通过LlamaIndex框架快速创建RAG应用的过程
- 了解RAG应用创建过程中的进阶用法


## 📖 课程目录

- [1. 计算环境准备](#💻-1-计算环境准备)
- [2. 向通义千问提问](#💬-2-向通义千问提问)
- [3. 实现RAG问答](#📚-3-实现rag问答)
- [4. 开发RAG应用](#⚙️-4-开发rag应用)
    - [4.1. 保存与加载索引文件](#41-保存与加载索引文件)
    - [4.2. 修改prompt模板](#42-修改prompt模板)
    - [4.3. 选择召回文本段个数](#43-选择召回文本段个数)
    - [4.4. 理解ReRank和相似度阈值](#44-理解rerank和相似度阈值)
    - [4.5. 句子窗口检索](#45-句子窗口检索)
    - [4.6. 自动合并检索](#46-自动合并检索)   
- [拓展阅读](#拓展阅读)


## 💻 1. 计算环境准备
### 1.1. 安装依赖


In [1]:
! pip install -r requirements.txt

### 1.2. 导入依赖

In [3]:
import os
import getpass
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    SimpleDirectoryReader,
    PromptTemplate,
    get_response_synthesizer,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.dashscope import (
    DashScopeEmbedding,
    DashScopeTextEmbeddingModels,
    DashScopeTextEmbeddingType
)
from llama_index.core.node_parser import SentenceWindowNodeParser,HierarchicalNodeParser
from llama_index.core.schema import Document
from llama_index.llms.dashscope import DashScope
from llama_index.core.schema import TextNode,NodeRelationship, RelatedNodeInfo,Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor,MetadataReplacementPostProcessor
from llama_index.core.base.llms.types import MessageRole, ChatMessage
from llama_index.postprocessor.dashscope_rerank import DashScopeRerank
from llama_index.core.response.notebook_utils import display_response,display_source_node
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.node_parser import get_leaf_nodes,get_root_nodes
from llama_index.core.storage.docstore import SimpleDocumentStore

### 1.3. 设置环境变量
运行下方程序，输入api_key即可

In [3]:
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("请输入你的api_key:")

# 导入key
import dashscope
dashscope.api_key = os.environ["DASHSCOPE_API_KEY"]

## 💬 2. 向通义千问提问
此处向你介绍通过LlamaIndex向通义千问模型提问的方法。

In [3]:
# 定义 LLM_MODEL
LLM_MODEL = DashScope(model_name="qwen-plus")
# 定义输入到 LLM_MODEL中的messages，你可以在此定义system message与user message
messages = [ChatMessage(role=MessageRole.SYSTEM, content="你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。"),
            ChatMessage(role=MessageRole.USER, content="你好")]

### 2.1. 非流式输出

你可以使用chat方法来进行非流式输出。

In [4]:
# 在Jupyter中，不用print就可以展示单一变量
LLM_MODEL.chat(messages).message.content

'你好！有什么问题我可以帮助你解答吗？无论是课程内容、学习方法还是其他相关问题，都欢迎提问。'

### 2.2. 流式输出

你可以使用stream_chat方法进行流式输出，大模型会一段段地将返回结果。

In [5]:
responses = LLM_MODEL.stream_chat(messages)
for response in responses:
    print(response.delta, end="")

你好！有什么问题我可以帮助你解答吗？无论是课程内容、学习方法还是其他相关问题，都欢迎提问。

## 📚 3. 实现RAG问答

LlamaIndex 对于RAG问答的创建与使用十分友好。你可以参考以下内容了解通过LlamaIndex进行RAG问答的基本步骤与更多使用方法。
### 3.1. 设置RAG的默认大模型与embedding模型
LlamaIndex有许多方法都需要指定大模型与embedding模型，但是一个项目里往往只需要用到一个大模型与一个embedding模型，因此你可以通过Settings设置默认的大模型与嵌入模型，设置后就无需在llamaindex的后续操作中再指定了。

In [6]:
def set_default_model():
    EMBED_MODEL = DashScopeEmbedding(
        # 你也可以使用其它通义千问embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2,
        text_type=DashScopeTextEmbeddingType.TEXT_TYPE_DOCUMENT,
    )
    # 你也可以使用其它通义千问大语言模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    LLM_MODEL = DashScope(model_name="qwen-plus")
    Settings.embed_model = EMBED_MODEL
    Settings.llm = LLM_MODEL

set_default_model()

### 3.2. 将文件夹中的文件解析为document对象
LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，而后续的索引创建需要基于document对象。将所有document的文本集成到一个总的document中。

In [7]:
def get_documents(path):
    documents = SimpleDirectoryReader(path).load_data()
    document = Document(text="\n\n".join([doc.text for doc in documents]))
    # 将中文的标点符号预先转化为英文标点符号，使得切分时不会使得chunk长度过长
    replacements = {
        '。': '. ',
        '！': '! ',
        '？': '? '
    }
    for src, dst in replacements.items():
        document.text = document.text.replace(src, dst)
    return document

# 获取docs文件夹中的document
document = get_documents(path="docs")

### 3.3. 一般的知识库问答
如果你需要通过LlamaIndex创建RAG应用，那么大致流程为：
1. 将解析好的文本转化为document对象；
2. 将document进行文本切分，返回node对象，每一个node对象都存放了切分好的文本段；
3. 将node加载为index（索引）；
4. 使用index.as_query_engine方法得到提问引擎，对提问引擎调用query方法进行提问。


此处展示了从创建索引-创建提问引擎-生成回复的流程。
> from_documents方法涵盖了第2步和第3步。

In [8]:
print("正在创建索引...")
# 这一步也会进行文本段的切分过程，此处使用默认切分方法
index = VectorStoreIndex.from_documents([document])
print("正在创建提问引擎...")
query_engine = index.as_query_engine(streaming=True)
print("正在生成回复...")
streaming_response = query_engine.query('需求分析使用的工具是什么？')
print("回答是：\n")
streaming_response.print_response_stream()

正在创建索引...
正在创建提问引擎...
正在生成回复...
回答是：

需求分析使用的工具包括项目管理软件如 Jira 和 Trello，文档编辑器如 Google Docs 和 Notion，以及协作工具如 Slack 和 Microsoft Teams。

## ⚙️ 4. 开发RAG应用

### 4.1. 保存与加载索引文件
1. 将索引保存到本地

你可以把当前创建的索引保存到本地，这样之后使用索引只要直接加载就可以了。

In [9]:
# 将索引保存为本地文件
index.storage_context.persist("knowledge_base/test")

2. 加载本地索引

加载索引的方法也很容易，请参考以下代码。

In [10]:
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context)

3. 查看召回文本段及其相似度分数
   
在向大模型发出提问后，返回的response对象中会包含source_nodes属性，你可以从中提取召回的文本段与对应的相似度分数。

In [11]:
def show_chunk_score(response):
    for i in range(len(response.source_nodes)):
        source_node = response.source_nodes[i]
        display_source_node(source_node=source_node,source_length=300)
        print("="*100)

show_chunk_score(streaming_response)

**Node ID:** c4488526-29e2-4599-a1f7-de54774e6b7a<br>**Similarity:** 0.4385171583575057<br>**Text:** 这意味着我需要不断学习这些平台的最新功能和用户体验设计. 

6. 教师培训与支持

为了确保最终用户能够有效使用我们提供的教育内容，我将参与制定教师培训计划，设计培训材料，并为教师提供持续的技术支持. 我会通过组织线上或线下的培训工作坊，让教师熟悉内容的使用方法，以及如何结合他们的教学实践来最大化学习效果. 

7. 数据分析与评估

最后，我会定期进行数据分析，以评估我们的教育内容的有效性与影响力. 通过分析学习者的学习数据、反馈和成绩，我可以直接了解内容的实际效果，并为未来的改进提供有力依据. 这种基于数据的决策过程将确保我们的教育内容始终与学习者的需求保持一致. 



工作流程...<br>

**Node ID:** fa72427d-5088-4579-a940-318f8da27c54<br>**Similarity:** 0.2335440110483407<br>**Text:** 内容开发工程师

岗位类型

大类：技术大类

细分类型：综合技术岗位

工作职责

核心职责

结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展. 

详细职责

1. 内容研究与分析

对最新的教育技术趋势、学习理论和市场需求进行深入研究. 这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中. 通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求. 

2. 教材和课程开发

根据研究和市场反馈，我将设计和开发高质量的教育教材和课程. 这包括撰写教学大纲...<br>

### 4.2. 修改prompt模板

在之前的教程中，我们使用的是LlamaIndex的默认英文提示词模板，模型有时可能会输出英文。在中文问答场景中，中文的提示词模板效果会更好。你可以通过以下方法学习修改prompt模板的方法。

In [12]:
# 查看原始模板
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].default_template.template)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


context_str为召回的文本段，query_str为用户的输入问题。

你可以通过以下方法，将prompt更换为中文，且可以添加人设等信息，使得回答更个性化。

In [13]:
# 修改prompt模板
def update_prompt_template(query_engine):
    # 修改prompt模板
    qa_prompt_tmpl_str = (
        "你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。以下是参考信息"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "你一定要参考提供的参考信息，而不是你之前就有的知识。"
        "以友好、和善的语气回答学员的提问。在结束回答后要感谢学员的提问。"
        "学员的提问是: {query_str}\n"
        "你的回答是: "
    )
    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
    )
    return query_engine

In [14]:
query_engine = index.as_query_engine(streaming=True)
query_engine = update_prompt_template(query_engine)
#查看当前的prompt
print("当前prompt模板为：")
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].template)
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
print("\n")
# 打印召回文本段与相似度分数
# show_chunk_score(response=response)

当前prompt模板为：
你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。以下是参考信息---------------------
{context_str}
---------------------
你一定要参考提供的参考信息，而不是你之前就有的知识。以友好、和善的语气回答学员的提问。在结束回答后要感谢学员的提问。学员的提问是: {query_str}
你的回答是: 
需求分析阶段使用的工具主要包括项目管理软件如Jira或Trello，这些工具可以帮助我们更好地跟踪需求的进展；文档编辑器如Google Docs或Notion，用于详细记录和整理需求文档；此外，还会用到协作工具如Slack或Microsoft Teams，以便于团队成员之间以及与利益相关者进行高效沟通。

感谢您的提问！如果您还有其他疑问，请随时告诉我。



### 4.3. 选择召回文本段个数
召回文本段的个数决定了大模型可以参考信息的多少。你可以尝试自定义召回的文本段个数，来找到最适合的值。

In [15]:
query_engine = index.as_query_engine(
    # 设置召回文本段个数
    similarity_top_k=3,
    streaming=True
)
query_engine = update_prompt_template(query_engine)  
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
print("\n")
# 打印召回文本段与相似度分数
# show_chunk_score(response)

需求分析阶段使用的工具主要包括：

- 项目管理软件：如Jira、Trello，帮助我们更好地规划和追踪项目进度。
- 文档编辑器：如Google Docs、Notion，便于撰写和共享详细的需求文档。
- 协作工具：如Slack、Microsoft Teams，促进团队成员间的沟通与协作。

这些工具可以帮助我们确保需求明确、沟通顺畅，并且按照SMART原则（具体、可测量、可实现、相关性、时限）来定义需求。感谢您的提问！如果您还有其他疑问，请随时告诉我。



### 4.4. 理解ReRank和相似度阈值
向量数据库检索出来的文本段并不总是最相关的，相关的文本段可能被排到了靠后的位置，并且召回的文本段可能与你提问的问题并不相关。<br>
你可以通过rerank（重排）模型与设置相似度阈值来解决这一问题。
- rerank<br>
  一般的步骤为：先检索较大数量的文本段，再通过rerank模型进行重排序，获得更精准的排名。
- 相似度阈值<br>
  通过对已召回文本段设置相似度阈值，过滤掉相似度低于阈值的文本段。

In [16]:
query_engine = index.as_query_engine(
    # 先设置一个较大的值
    similarity_top_k=8,
    streaming=True,
    node_postprocessors=[
        # 在rerank模型中选择你想召回的文本段个数，重排模型选择通义实验室的gte-rerank模型
        DashScopeRerank(top_n=3, model="gte-rerank"),
        # 设置一个相似度阈值，低于该阈值的文本段会被过滤掉
        SimilarityPostprocessor(similarity_cutoff=0.2)])
# 更新提示词模板
query_engine = update_prompt_template(query_engine)
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
# 打印召回文本段与相似度分数
# show_chunk_score(response)

需求分析阶段使用的工具主要包括项目管理软件（如 Jira、Trello）、文档编辑器（如 Google Docs、Notion）以及协作工具（如 Slack、Microsoft Teams）。这些工具可以帮助我们更好地组织和管理需求信息，促进团队间的沟通与协作，确保需求被准确地理解和实现。

感谢您的提问！如果您还有其他疑问，请随时向我询问。

### 4.5. 句子窗口检索
句子窗口检索是一种能够在检索阶段排除噪声干扰信息，在生成阶段补充信息的方法。具体步骤如下：
1. 将文本按照"."、"?"、"!"切分成chunk；
2. 设置window_size参数，构建K-V对。K值为第一步切分的chunk；V值为以该chunk为中心，向左右两侧各寻找window_size个chunk。
3. 在检索时，用query与第一步切分的chunk对比，将检索出的chunk对应的V值取出，通过后处理的模块将其作为生成时的参考内容。


总结起来就是：**句子窗口检索**通过先检索出最相关的句子，然后返回围绕该句子的更广泛的文本，为生成阶段提供更丰富的上下文，从而提高生成内容的相关性和准确性。

In [17]:
node_parser = SentenceWindowNodeParser.from_defaults(
    # 在这里设置窗口大小
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents([document])

1. 在进行句子滑窗检索前（没有开启后处理）

In [18]:
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(similarity_top_k=1)
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

**`Final Response:`** 在开发过程中，应注意定期备份内容，以防止数据丢失。

In [19]:
show_chunk_score(response)

**Node ID:** 7e9e97b9-d0fc-4333-8630-20eb1b995200<br>**Similarity:** 0.4507105722309893<br>**Text:** 定期备份开发中的内容，以防数据丢失.<br>

2. 进行句子滑窗后（开启后处理模块）

In [20]:
window_text_replace = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)
query_engine = index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[window_text_replace])
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

**`Final Response:`** 开发阶段需要注意以下事项：确保内容符合教育标准和教学目标，使用规范的术语和格式以保持内容的一致性，定期备份内容以防止数据丢失。

In [21]:
show_chunk_score(response)

**Node ID:** 7e9e97b9-d0fc-4333-8630-20eb1b995200<br>**Similarity:** 0.4507105722309893<br>**Text:** 与团队分享需求文档，收集反馈并进行调整. 



 开发



使用工具：

内容创作工具（Markdown Editor、Adobe Creative Suite）

版本控制系统（Git）

教育平台（Moodle、Blackboard）

注意事项：

确保内容符合教育标准和教学目标. 

 使用规范的术语和格式，保持内容一致性. 

 定期备份开发中的内容，以防数据丢失. 

 操作指导：

根据需求文档创建内容框架. 

 编写、设计或录制相关教育内容. 

 定期在版本控制系统中提交更新，记录更改.<br>

### 4.6. 自动合并检索
与句子窗口检索类似，自动合并检索也是先检索出最相关的句子，然后返回围绕该句子的更广泛的文本。它在切割文本段的时候会按照定义的层级结构进行切分。

假设chunk_sizes=[512, 128]，那么在切割文本段时，叶节点每一个chunk的最长字符串长度为128；父节点会包含它的叶节点，每一个chunk的最长字符串长度为512。

在检索时，自动合并检索方法会按照叶节点进行检索，然后对父节点进行统计：如果某一父节点的多数叶节点（比例可以设置，通过simple_ratio_thresh参数设置）被召回，那么就把该父节点作为上下文传入大模型；如果每一个父节点都没有多数叶节点被召回，那么就把单独的叶节点作为上下文传入大模型。

In [22]:
# 使用默认设置创建层次结构节点解析器
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[512, 128]
)
nodes = node_parser.get_nodes_from_documents([document])

In [23]:
# 获取叶节点并打印其中一个的文本作为示例
leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[1].text)

详细职责

1. 内容研究与分析

对最新的教育技术趋势、学习理论和市场需求进行深入研究. 这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中.


In [24]:
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [25]:
index = VectorStoreIndex(leaf_nodes)
base_retriever = index.as_retriever(similarity_top_k=7)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)
query_engine = RetrieverQueryEngine.from_args(retriever)
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

**`Final Response:`** 开发阶段的注意事项包括确保内容符合教育标准和教学目标，使用规范的术语和格式以保持内容的一致性，以及定期备份开发中的内容以防数据丢失。

In [26]:
show_chunk_score(response)

**Node ID:** e8bc56f7-6b33-4807-a016-8259485aad3a<br>**Similarity:** 0.4912935291233606<br>**Text:** 收集用户反馈和需求，确保满足目标受众的期望. 

操作指导：

召开需求沟通会议，记录会议纪要. 

形成需求文档，描述每个功能或内容的细节. 

与团队分享需求文档，收集反馈并进行调整. 



开发<br>

**Node ID:** 4737d595-4602-4633-b991-87185416693d<br>**Similarity:** 0.4046938714757357<br>**Text:** 开发



使用工具：

内容创作工具（Markdown Editor、Adobe Creative Suite）

版本控制系统（Git）

教育平台（Moodle、Blackboard）

注意事项：

确保内容符合教育标准和教学目标. 

使用规范的术语和格式，保持内容一致性. 

定期备份开发中的内容，以防数据丢失. 

操作指导：

根据需求文档创建内容框架.<br>

**Node ID:** 6aa34f49-6a01-4fa4-9190-8a2f576d0b79<br>**Similarity:** 0.3614180518042455<br>**Text:** 内容开发工程师

岗位类型

大类：技术大类

细分类型：综合技术岗位

工作职责

核心职责

结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展. 

详细职责

1.<br>

**Node ID:** 4d59b4fc-7aa5-45ba-bdb2-771924ae1b94<br>**Similarity:** 0.35943489810687407<br>**Text:** 工作流程指导

需求分析

使用工具：

项目管理软件（Jira、Trello）

文档编辑器（Google Docs、Notion）

协作工具（Slack、Microsoft Teams）

注意事项：

确保需求明确，遵循SMART原则（具体、可测量、可实现、相关性、时限）. 

与相关利益相关者进行充分沟通，确认需求的优先级.<br>

**Node ID:** 8c029123-1b07-4e63-a166-788c1d754711<br>**Similarity:** 0.34465970232956633<br>**Text:** 3. 内容优化与更新

在内容开发过程中，我会不断优化已有的教育材料.<br>

**Node ID:** a26f4935-c135-4c5d-bea0-d7ba7bbf3b8c<br>**Similarity:** 0.33834932305215243<br>**Text:** - 进行性能调优或重构代码. 

     

以上处理步骤有助于迅速恢复内容平台的正常运行，减少对用户体验的影响.<br>

**Node ID:** 17ec717b-f5eb-4c3b-bf76-a5c167dc712c<br>**Similarity:** 0.3339497691017454<br>**Text:** 操作指导：

在内容管理系统中输入最终版本，设置发布参数. 

进行最终审查，确保无遗漏和错误. 

执行发布操作，并监测发布状态.<br>

## 拓展阅读

如果你想要创建以输入框形式交互的RAG应用，请阅读：[百炼RAG实践教程文档](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval)。

<img src="https://help-static-aliyun-doc.aliyuncs.com/assets/img/zh-CN/4139126271/p844266.gif" alt="我的notebook" width="600px">

你可以在下载[文档中的压缩包](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval#b91232e354cb4)后，找到chat.py中的get_model_response函数并进行修改，适配到本教程介绍的方法。

## ✅ 本节小结
通过学习本节课程，你已经掌握了LlamaIndex的基本用法。

## 🔥 课后小测验


【单选题】 2.1.1. 以下代码片段的作用是什么？（ ）
```python
index.storage_context.persist(persist_dir=PERSIST_DIR)
```
A. 从磁盘加载嵌入向量。

B. 将嵌入向量存储到内存中。

C. 将嵌入向量持久化到指定的目录。

D. 创建一个新的 StorageContext 对象。

答案：C